In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import os
import json

In [3]:
def get_exact_time_lag_fast(df, value_column, timestamp_column='timestamp_utc', lag_hours=168):
    """
    Optimized version of get_exact_time_lag using vectorized operations.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the time series data
    value_column : str
        Name of the column containing values to be lagged
    timestamp_column : str
        Name of the column containing timestamps
    lag_hours : int
        Number of hours to look back for the lag
    
    Returns:
    --------
    pandas.Series
        Series containing the lagged values
    """
    # Calculate target timestamps
    target_timestamps = df[timestamp_column] - pd.Timedelta(hours=lag_hours)
    
    # Create a merged dataframe to find matches
    reference_df = pd.DataFrame({
        'reference_time': df[timestamp_column],
        'value': df[value_column]
    }).sort_values('reference_time')
    
    # Use searchsorted to find the insertion points
    idx = np.searchsorted(reference_df['reference_time'], target_timestamps, side='right') - 1
    
    # Handle cases where idx is -1 (target time before any reference time)
    idx = np.where(idx < 0, 0, idx)
    
    # Get the matched values
    return reference_df['value'].iloc[idx].values

In [4]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
df_bbidding["timestamp_utc"] = pd.to_datetime(df_bbidding["timestamp_utc"])

In [5]:
# Modified code to apply to your DataFrame
df_bbidding1 = df_bbidding[["timestamp_utc","market_price","day_ahead_price","volume",
                           "settlement_period","cos_hour","cos_day","1","2","3","4","5",
                           "6","7","8","9","imbalance_price"]].copy()

lag_configs = {
    "market_price_lag96h": ("market_price", 96),
    "imbalance_price_lag96h": ("imbalance_price", 96),
    "volume_lag96h": ("volume", 96),
    "day_ahead_price_lag1week": ("day_ahead_price", 168)
}

for new_col, (source_col, hours) in lag_configs.items():
    df_bbidding1[new_col] = get_exact_time_lag_fast(
        df_bbidding1, 
        value_column=source_col,
        lag_hours=hours
    )

In [6]:
# dfbidding1 drop first week
df_bbidding1 = df_bbidding1.loc[df_bbidding1["timestamp_utc"] > "2021-02-26 23:00:00+00:00"]

In [7]:
# df_bbidding1 = df_bbidding[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
# df_bbidding1.loc[:,"market_price_lag96h"] = df_bbidding1["market_price"].shift(192)
# df_bbidding1.loc[:,"imbalance_price_lag96h"] = df_bbidding1["imbalance_price"].shift(192)
# df_bbidding1.loc[:,"day_ahead_price_lag1week"] = df_bbidding1["day_ahead_price"].shift(336)
# df_bbidding1.loc[:,"volume_lag96h"] = df_bbidding1["volume"].shift(192)
# df_bbidding1.dropna(inplace=True)
# df_bbidding1

In [8]:
10 davor -> nächster wert

SyntaxError: invalid syntax (1920019420.py, line 1)

In [9]:
# Original dataset
X = df_bbidding1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
y = df_bbidding1["day_ahead_price"].values  # Convert to numpy

# Step 1: Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 2: Further split the training set into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Step 3: Standardize the data (use only training data to fit the scaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [10]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [11]:
# y_train = y_train.values
# y_test = y_test.values

In [12]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Step 4: Create TensorDataset and DataLoader for training, validation, and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Hyperparameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1                  # Dropout rate
learning_rate = 0.0001         # Learning rate for optimizer
batch_size = 32                # Batch size
num_epochs = 500               # Maximum number of epochs
patience = 15                  # Patience for early stopping
rel_improvement_threshold = 0.00005  # Relative improvement threshold

In [13]:
model = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout)

# Loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

best_loss = float('inf')
best_model = None
patience_counter = 0
previous_loss = float('inf')

In [14]:
import torch.nn.functional as F  # For MAE loss function

best_val_loss = float('inf')  # Initialize the best validation loss
best_model = None
patience_counter = 0  # For early stopping

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0

    # Training loop
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension (for some models)
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the loss using MAE
        loss = F.l1_loss(y_pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    epoch_train_loss = running_train_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}')

    # Validation loop
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Reshape input to add sequence length dimension
            X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

            # Forward pass
            y_pred = model(X_batch)  # (batch_size, output_size)

            # Compute the validation loss using MAE
            val_loss = F.l1_loss(y_pred, y_batch)
            running_val_loss += val_loss.item()

    epoch_val_loss = running_val_loss / len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

    # Early stopping based on validation loss
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        best_model = model.state_dict()  # Save the best model's weights
        patience_counter = 0  # Reset patience counter
        print(f"Best model saved with validation loss: {best_val_loss:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

# Load the best model
if best_model is not None:
    model.load_state_dict(best_model)
else:
    print("No improvement was observed during training.")

torch.save(model.state_dict(), 'LSTM_day_ahead_price.pth')

# Test the model using the best model
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the test loss using MAE
        loss = F.l1_loss(y_pred, y_batch)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_loader):.4f}')


Epoch [1/500], Training Loss: 166.6196
Epoch [1/500], Validation Loss: 142.5634
Best model saved with validation loss: 142.5634
Epoch [2/500], Training Loss: 157.6807
Epoch [2/500], Validation Loss: 136.9112
Best model saved with validation loss: 136.9112
Epoch [3/500], Training Loss: 152.7931
Epoch [3/500], Validation Loss: 132.5481
Best model saved with validation loss: 132.5481
Epoch [4/500], Training Loss: 148.4557
Epoch [4/500], Validation Loss: 128.4445
Best model saved with validation loss: 128.4445
Epoch [5/500], Training Loss: 144.2725
Epoch [5/500], Validation Loss: 124.4585
Best model saved with validation loss: 124.4585
Epoch [6/500], Training Loss: 140.1564
Epoch [6/500], Validation Loss: 120.5357
Best model saved with validation loss: 120.5357
Epoch [7/500], Training Loss: 136.0831
Epoch [7/500], Validation Loss: 116.6720
Best model saved with validation loss: 116.6720
Epoch [8/500], Training Loss: 132.0493
Epoch [8/500], Validation Loss: 112.8580
Best model saved with va

In [15]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [16]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)
df_api_new = df_api_new.groupby("timestamp").last().reset_index()
df_api_new

,timestamp,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-10-03T22:00:00+00:00,2024-10-04,1355,487,963,1544,1330,867,326,1402,1357
1,2024-10-03T22:30:00+00:00,2024-10-04,788,584,1148,1320,1007,565,923,797,487
2,2024-10-03T23:00:00+00:00,2024-10-04,571,748,1001,1460,740,1017,1533,597,991
3,2024-10-03T23:30:00+00:00,2024-10-04,1476,1194,1512,334,1054,1572,744,1497,463
4,2024-10-04T00:00:00+00:00,2024-10-04,1352,1390,912,662,1505,1093,814,487,1340
...,...,...,...,...,...,...,...,...,...,...,...
909,2024-10-27T20:30:00+00:00,2024-10-27,278,337,389,433,480,518,562,610,645
910,2024-10-27T21:00:00+00:00,2024-10-27,250,337,390,439,477,516,547,586,618
911,2024-10-27T21:30:00+00:00,2024-10-27,284,368,406,458,489,502,546,567,596
912,2024-10-27T22:00:00+00:00,2024-10-27,309,389,429,457,483,493,505,517,548


In [17]:
from datetime import datetime, timedelta
df_api_new = df_api_new.rename(columns={
    "timestamp": "datetime"
})
df_api_new["datetime"] = pd.to_datetime(df_api_new["datetime"])
min_date = df_api_new.datetime.min() - timedelta(minutes=30)
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
df_half_hourly = pd.DataFrame({"datetime": datetimes})
df_half_hourly["datetime"] = pd.to_datetime(df_half_hourly["datetime"])
df_api_new = pd.merge(df_half_hourly, df_api_new, left_on='datetime', right_on='datetime', how='outer')
df_api_new

,datetime,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-09-26 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-09-26 22:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-09-26 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-09-26 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-09-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1245,2024-10-27 20:30:00+00:00,2024-10-27,278.0,337.0,389.0,433.0,480.0,518.0,562.0,610.0,645.0
1246,2024-10-27 21:00:00+00:00,2024-10-27,250.0,337.0,390.0,439.0,477.0,516.0,547.0,586.0,618.0
1247,2024-10-27 21:30:00+00:00,2024-10-27,284.0,368.0,406.0,458.0,489.0,502.0,546.0,567.0,596.0
1248,2024-10-27 22:00:00+00:00,2024-10-27,309.0,389.0,429.0,457.0,483.0,493.0,505.0,517.0,548.0


In [18]:
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new,df_day_ahead_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.day/7)
df_api_new_merged

,datetime,prediction_date,1,2,3,4,5,6,7,8,...,price_y,volume,timestamp_utc,settlement_date,settlement_period,imbalance_price,day_ahead_price,market_price,cos_hour,cos_day
0,2024-09-26 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.21,1485.30,2024-09-26 22:00:00+00:00,2024-09-26,47.0,68.929966,7.20,21.21,0.866025,-0.222521
1,2024-09-26 22:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.46,1364.20,2024-09-26 22:30:00+00:00,2024-09-26,48.0,-28.480000,7.20,3.46,0.866025,-0.222521
2,2024-09-26 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.15,1688.95,2024-09-26 23:00:00+00:00,2024-09-27,1.0,-19.020000,3.87,3.15,0.965926,-0.222521
3,2024-09-26 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.68,1661.10,2024-09-26 23:30:00+00:00,2024-09-27,2.0,-18.000000,3.87,-3.68,0.965926,-0.222521
4,2024-09-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.52,1824.60,2024-09-27 00:00:00+00:00,2024-09-27,3.0,56.450000,0.45,-0.52,1.000000,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2024-10-27 20:30:00+00:00,2024-10-27,278.0,337.0,389.0,433.0,480.0,518.0,562.0,610.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,0.623490
1255,2024-10-27 21:00:00+00:00,2024-10-27,250.0,337.0,390.0,439.0,477.0,516.0,547.0,586.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.707107,0.623490
1256,2024-10-27 21:30:00+00:00,2024-10-27,284.0,368.0,406.0,458.0,489.0,502.0,546.0,567.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.707107,0.623490
1257,2024-10-27 22:00:00+00:00,2024-10-27,309.0,389.0,429.0,457.0,483.0,493.0,505.0,517.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.866025,0.623490


In [19]:
df_api_new_merged1 = df_api_new_merged[["datetime","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag96h"] = df_api_new_merged1["market_price"].shift(192)
df_api_new_merged1.loc[:,"imbalance_price_lag96h"] = df_api_new_merged1["imbalance_price"].shift(192)
df_api_new_merged1.loc[:,"day_ahead_price_lag1week"] = df_api_new_merged1["day_ahead_price"].shift(336)
df_api_new_merged1.loc[:,"volume_lag96h"] = df_api_new_merged1["volume"].shift(192)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1

,datetime,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h
336,2024-10-03 22:00:00+00:00,71.32,78.47,1322.45,47.0,0.866025,-0.900969,1355.0,487.0,963.0,...,1330.0,867.0,326.0,1402.0,1357.0,61.00,12.85,-1.4600,7.20,1457.25
337,2024-10-03 22:30:00+00:00,71.42,78.47,1035.65,48.0,0.866025,-0.900969,788.0,584.0,1148.0,...,1007.0,565.0,923.0,797.0,487.0,50.00,-3.95,-1.4600,7.20,1367.60
338,2024-10-03 23:00:00+00:00,66.94,71.36,1378.55,1.0,0.965926,-0.900969,571.0,748.0,1001.0,...,740.0,1017.0,1533.0,597.0,991.0,93.50,-2.74,5.0000,3.87,1473.20
339,2024-10-03 23:30:00+00:00,60.03,71.36,1539.90,2.0,0.965926,-0.900969,1476.0,1194.0,1512.0,...,1054.0,1572.0,744.0,1497.0,463.0,93.50,3.85,5.0000,3.87,1799.05
340,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,912.0,...,1505.0,1093.0,814.0,487.0,1340.0,93.00,0.03,-2.4304,0.45,1689.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,2024-10-21 19:30:00+00:00,71.47,79.79,2221.80,42.0,0.258819,1.000000,263.0,343.0,406.0,...,482.0,501.0,514.0,547.0,590.0,65.00,86.49,71.5200,91.30,2070.40
1010,2024-10-21 20:00:00+00:00,87.87,77.99,2156.50,43.0,0.500000,1.000000,266.0,345.0,405.0,...,485.0,504.0,516.0,551.0,592.0,99.00,81.51,68.0000,85.25,1475.55
1011,2024-10-21 20:30:00+00:00,86.70,77.99,2183.15,44.0,0.500000,1.000000,257.0,341.0,397.0,...,482.0,508.0,515.0,550.0,585.0,99.00,82.97,67.7000,85.25,1298.75
1012,2024-10-21 21:00:00+00:00,71.16,74.71,1754.15,45.0,0.707107,1.000000,249.0,338.0,389.0,...,478.0,508.0,514.0,553.0,587.0,0.94,80.73,70.5000,91.58,1364.60


In [20]:
X_test = df_api_new_merged1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
X_test = scaler.transform(X_test)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
X_test_tensor.shape

torch.Size([676, 15])

In [21]:
X_test_tensor = X_test_tensor.unsqueeze(1)  # Adds a sequence length dimension


In [22]:


model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_outputs = final_test_outputs.numpy().flatten()

df_api_new_merged1["day_ahead_price_predictions"] = final_test_outputs
df_api_new_merged1

,datetime,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h,day_ahead_price_predictions
336,2024-10-03 22:00:00+00:00,71.32,78.47,1322.45,47.0,0.866025,-0.900969,1355.0,487.0,963.0,...,867.0,326.0,1402.0,1357.0,61.00,12.85,-1.4600,7.20,1457.25,52.107071
337,2024-10-03 22:30:00+00:00,71.42,78.47,1035.65,48.0,0.866025,-0.900969,788.0,584.0,1148.0,...,565.0,923.0,797.0,487.0,50.00,-3.95,-1.4600,7.20,1367.60,56.578751
338,2024-10-03 23:00:00+00:00,66.94,71.36,1378.55,1.0,0.965926,-0.900969,571.0,748.0,1001.0,...,1017.0,1533.0,597.0,991.0,93.50,-2.74,5.0000,3.87,1473.20,60.872398
339,2024-10-03 23:30:00+00:00,60.03,71.36,1539.90,2.0,0.965926,-0.900969,1476.0,1194.0,1512.0,...,1572.0,744.0,1497.0,463.0,93.50,3.85,5.0000,3.87,1799.05,47.774483
340,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,912.0,...,1093.0,814.0,487.0,1340.0,93.00,0.03,-2.4304,0.45,1689.75,52.903481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,2024-10-21 19:30:00+00:00,71.47,79.79,2221.80,42.0,0.258819,1.000000,263.0,343.0,406.0,...,501.0,514.0,547.0,590.0,65.00,86.49,71.5200,91.30,2070.40,118.427765
1010,2024-10-21 20:00:00+00:00,87.87,77.99,2156.50,43.0,0.500000,1.000000,266.0,345.0,405.0,...,504.0,516.0,551.0,592.0,99.00,81.51,68.0000,85.25,1475.55,102.159332
1011,2024-10-21 20:30:00+00:00,86.70,77.99,2183.15,44.0,0.500000,1.000000,257.0,341.0,397.0,...,508.0,515.0,550.0,585.0,99.00,82.97,67.7000,85.25,1298.75,103.556618
1012,2024-10-21 21:00:00+00:00,71.16,74.71,1754.15,45.0,0.707107,1.000000,249.0,338.0,389.0,...,508.0,514.0,553.0,587.0,0.94,80.73,70.5000,91.58,1364.60,109.166946


In [23]:
#plot imbalance price and predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_api_new_merged1["datetime"], y=df_api_new_merged1["day_ahead_price"], mode='lines', name='Imbalance Price'))
fig.add_trace(go.Scatter(x=df_api_new_merged1["datetime"], y=df_api_new_merged1["day_ahead_price_predictions"], mode='lines', name='Imbalance Price Predictions'))
fig.update_layout(title='Imbalance Price and Predictions', xaxis_title='Timestamp', yaxis_title='Imbalance Price')
fig.show()

In [24]:
#MAE calculation for imbalance price predictions
mae = np.mean(np.abs(df_api_new_merged1["day_ahead_price"] - df_api_new_merged1["day_ahead_price_predictions"]))
print(f"MAE for imbalance price predictions: {mae:.4f}")

MAE for imbalance price predictions: 34.9381


In [25]:
df_api_new_merged1.drop(columns=["datetime"]).corr()["day_ahead_price"]

market_price                   0.459050
day_ahead_price                1.000000
volume                         0.023106
settlement_period              0.237255
cos_hour                      -0.332983
cos_day                       -0.124310
1                             -0.084179
2                             -0.124610
3                             -0.120535
4                             -0.126306
5                             -0.131031
6                             -0.115995
7                             -0.120230
8                             -0.105632
9                             -0.107018
imbalance_price                0.572484
market_price_lag96h           -0.013696
imbalance_price_lag96h         0.100072
day_ahead_price_lag1week       0.128432
volume_lag96h                  0.139725
day_ahead_price_predictions    0.187751
Name: day_ahead_price, dtype: float64

In [26]:
df_day_ahead_demand = pd.read_csv('D:/Users/paulh\Desktop/Domäneprojekt2/Energy_production_price_prediction/day_ahead_demand_forecast.csv')
df_margin_forecast = pd.read_csv('D:/Users/paulh\Desktop/Domäneprojekt2/Energy_production_price_prediction/margin_forecast.csv')
df_margin_forecast.forecast_date = pd.to_datetime(df_margin_forecast.forecast_date)
df_day_ahead_demand.timestamp_utc = pd.to_datetime(df_day_ahead_demand.timestamp_utc)
df_api_new_merged2 = pd.merge(df_api_new_merged1,df_day_ahead_demand, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged2["date"] = df_api_new_merged2["datetime"].dt.date
df_api_new_merged2["date"] = pd.to_datetime(df_api_new_merged2["date"])
df_api_new_merged2 = pd.merge(df_api_new_merged2,df_margin_forecast, left_on='date', right_on='forecast_date', how='left')
df_api_new_merged2.dropna(inplace=True)
df_api_new_merged2 = df_api_new_merged2.groupby("timestamp_utc").last().reset_index()
df_api_new_merged2

,timestamp_utc,datetime,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,...,settlement_date,settlement_period_y,boundary,publish_time_utc_x,transmission_system_demand,national_demand,date,forecast_date,publish_time_utc_y,margin
0,2024-10-04 00:00:00+00:00,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,...,2024-10-04,3,N,2024-10-03T20:45:00Z,22051.0,21551.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
1,2024-10-04 00:30:00+00:00,2024-10-04 00:30:00+00:00,64.79,72.80,1784.30,4.0,1.000000,-0.900969,1279.0,410.0,...,2024-10-04,4,N,2024-10-03T20:45:00Z,21886.0,21386.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
2,2024-10-04 01:00:00+00:00,2024-10-04 01:00:00+00:00,64.26,65.30,1647.20,5.0,0.965926,-0.900969,1223.0,378.0,...,2024-10-04,5,N,2024-10-04T00:46:00Z,21486.0,20986.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
3,2024-10-04 01:30:00+00:00,2024-10-04 01:30:00+00:00,65.61,65.30,1406.60,6.0,0.965926,-0.900969,848.0,1046.0,...,2024-10-04,6,N,2024-10-04T00:46:00Z,21471.0,20971.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
4,2024-10-04 02:00:00+00:00,2024-10-04 02:00:00+00:00,58.19,71.46,1149.45,7.0,0.866025,-0.900969,1469.0,543.0,...,2024-10-04,7,N,2024-10-04T00:46:00Z,20947.0,20447.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,2024-10-21 19:30:00+00:00,2024-10-21 19:30:00+00:00,71.47,79.79,2221.80,42.0,0.258819,1.000000,263.0,343.0,...,2024-10-21,42,N,2024-10-21T18:15:00Z,32190.0,30136.0,2024-10-21,2024-10-21,2024-10-19T21:00:00Z,16904.0
662,2024-10-21 20:00:00+00:00,2024-10-21 20:00:00+00:00,87.87,77.99,2156.50,43.0,0.500000,1.000000,266.0,345.0,...,2024-10-21,43,N,2024-10-21T18:15:00Z,30604.0,28749.0,2024-10-21,2024-10-21,2024-10-19T21:00:00Z,16904.0
663,2024-10-21 20:30:00+00:00,2024-10-21 20:30:00+00:00,86.70,77.99,2183.15,44.0,0.500000,1.000000,257.0,341.0,...,2024-10-21,44,N,2024-10-21T18:15:00Z,29396.0,27437.0,2024-10-21,2024-10-21,2024-10-19T21:00:00Z,16904.0
664,2024-10-21 21:00:00+00:00,2024-10-21 21:00:00+00:00,71.16,74.71,1754.15,45.0,0.707107,1.000000,249.0,338.0,...,2024-10-21,45,N,2024-10-21T20:45:00Z,27294.0,26052.0,2024-10-21,2024-10-21,2024-10-19T21:00:00Z,16904.0


In [27]:
df_api_new_merged2_train = df_api_new_merged2[:400]
df_api_new_merged2_test = df_api_new_merged2[400:]

In [28]:
import torch
import torch.nn as nn

class SimpleModel(nn.Module):
    def __init__(self, input_dim):
        super(SimpleModel, self).__init__()
        
        # Zwei Hidden Layers
        self.fc1 = nn.Linear(input_dim, 128)  # Erster Hidden Layer
        self.fc2 = nn.Linear(128, 64)         # Zweiter Hidden Layer
        
        # Ausgangsschicht
        self.fc3 = nn.Linear(64, 1)           # Ausgangsschicht
        
        # Dropout und Aktivierungsfunktion
        self.dropout = nn.Dropout(0.2)
        self.swish = nn.SiLU()

    def forward(self, x):
        # Durch die Hidden Layers
        x = self.swish(self.fc1(x))  # Erster Hidden Layer
        x = self.dropout(x)
        x = self.swish(self.fc2(x))  # Zweiter Hidden Layer
        x = self.dropout(x)
        
        # Ausgang
        x = self.fc3(x)  # Ausgangsschicht
        
        return x


In [29]:
cobined_X_train = df_api_new_merged2_train[["day_ahead_price_predictions","national_demand","transmission_system_demand","margin"]]
cobined_y_train = df_api_new_merged2_train["day_ahead_price"]

cobined_X_test = df_api_new_merged2_test[["day_ahead_price_predictions","national_demand","transmission_system_demand","margin"]]
cobined_y_test = df_api_new_merged2_test["day_ahead_price"]

cobined_X_train = scaler.fit_transform(cobined_X_train)
cobined_X_test = scaler.transform(cobined_X_test)

with open('scaler_LSTM_extension_day_ahead.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [35]:
combined_X_train_tensor = torch.tensor(cobined_X_train, dtype=torch.float32)
combined_y_train_tensor = torch.tensor(cobined_y_train.values, dtype=torch.float32).view(-1, 1)
combined_X_test_tensor = torch.tensor(cobined_X_test, dtype=torch.float32)
combined_y_test_tensor = torch.tensor(cobined_y_test.values, dtype=torch.float32).view(-1, 1)

# Step 4: Create TensorDataset and DataLoader for training and testing
combined_train_dataset = TensorDataset(combined_X_train_tensor, combined_y_train_tensor)
combined_test_dataset = TensorDataset(combined_X_test_tensor, combined_y_test_tensor)

# DataLoader
combined_train_loader = DataLoader(dataset=combined_train_dataset, batch_size=16, shuffle=False)
combined_test_loader = DataLoader(dataset=combined_test_dataset, batch_size=16, shuffle=False)

# Step 5: Initialize the model, loss function, and optimizer
input_dim = combined_X_train_tensor.shape[1]
model = SimpleModel(input_dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define MAE loss
mae_loss = nn.L1Loss()

# Training parameters
num_epochs = 100

In [36]:
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in combined_train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = mae_loss(outputs, y_batch)  # Use MAE loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_train_loss = epoch_loss / len(combined_train_loader)
    train_losses.append(avg_train_loss)
    
    # Test phase
    model.eval()
    with torch.no_grad():
        test_outputs = model(combined_X_test_tensor)
        test_loss = mae_loss(test_outputs, combined_y_test_tensor).item()  # Use MAE loss
        test_losses.append(test_loss)
    
    # Print progress
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {avg_train_loss:.4f}, "
              f"Test Loss: {test_loss:.4f}")
        
torch.save(model.state_dict(), 'MLP_extension_day_ahead.pth')

# Final evaluation
model.eval()
with torch.no_grad():
    final_test_outputs = model(combined_X_test_tensor)
    final_test_loss = mae_loss(final_test_outputs, combined_y_test_tensor).item()  # Use MAE loss
print(f"Final Test Loss: {final_test_loss:.4f}")


Epoch [5/200], Train Loss: 81.1848, Test Loss: 73.7755
Epoch [10/200], Train Loss: 80.6641, Test Loss: 73.2530
Epoch [15/200], Train Loss: 79.7395, Test Loss: 72.3625
Epoch [20/200], Train Loss: 78.2762, Test Loss: 70.9088
Epoch [25/200], Train Loss: 75.9477, Test Loss: 68.7276
Epoch [30/200], Train Loss: 72.8384, Test Loss: 65.7434
Epoch [35/200], Train Loss: 68.9555, Test Loss: 62.0114
Epoch [40/200], Train Loss: 63.6033, Test Loss: 57.7158
Epoch [45/200], Train Loss: 57.9562, Test Loss: 52.9552
Epoch [50/200], Train Loss: 51.2455, Test Loss: 47.5437
Epoch [55/200], Train Loss: 43.5449, Test Loss: 41.4303
Epoch [60/200], Train Loss: 36.1004, Test Loss: 35.1194
Epoch [65/200], Train Loss: 29.9235, Test Loss: 30.3295
Epoch [70/200], Train Loss: 26.7342, Test Loss: 26.9292
Epoch [75/200], Train Loss: 24.6342, Test Loss: 24.7825
Epoch [80/200], Train Loss: 21.5888, Test Loss: 23.1763
Epoch [85/200], Train Loss: 21.3975, Test Loss: 22.3179
Epoch [90/200], Train Loss: 19.7836, Test Loss: 2

In [37]:
model.eval()
with torch.no_grad():
    final_test_outputs = model(combined_X_test_tensor)
    final_test_outputs = final_test_outputs.numpy().flatten()

df_api_new_merged2_test["day_ahead_price_predictions_2"] = final_test_outputs

C:\Users\paulh\AppData\Local\Temp\ipykernel_14460\421217961.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
#plot imbalance price and predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_api_new_merged2_test["timestamp_utc"], y=df_api_new_merged2_test["day_ahead_price"], mode='lines', name='day_ahead_price'))
fig.add_trace(go.Scatter(x=df_api_new_merged2_test["timestamp_utc"], y=df_api_new_merged2_test["day_ahead_price_predictions_2"], mode='lines', name='day_ahead_price Predictions'))

In [41]:
#MAE calculation for imbalance price predictions
mae = np.mean(np.abs(df_api_new_merged2_test["day_ahead_price"] - df_api_new_merged2_test["day_ahead_price_predictions_2"]))
mae

21.231153715893733

In [40]:
test

NameError: name 'test' is not defined

In [103]:
#prediction for the whole dataset
X = df_bbidding1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
X = scaler.transform(X)
X_tensor = torch.tensor(X, dtype=torch.float32)
X_tensor = X_tensor.unsqueeze(1)  # Adds a sequence length

model.eval()
with torch.no_grad():
    final_outputs = model(X_tensor)
    final_outputs = final_outputs.numpy().flatten()

df_bbidding1["day_ahead_price_predictions"] = final_outputs
df_bbidding1

C:\Users\paulh\AppData\Local\Temp\ipykernel_15456\4141625092.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,volume_lag96h,day_ahead_price_lag1week,day_ahead_price_predictions
146,2021-02-28 00:00:00+00:00,55.58,53.03,497.60,1,1.000000,0.527078,3.263746e-07,-4.324696,-0.385406,...,2.732664e-20,-0.742440,1.315346,22.304271,33.65,-9.71,7.00,900.35,17.34,43.163513
147,2021-02-28 00:30:00+00:00,57.95,53.03,554.50,2,1.000000,0.527078,3.263746e-07,-4.324696,-0.385363,...,2.732664e-20,-0.736182,1.315346,22.306173,62.70,-9.71,7.00,900.35,17.34,43.163528
148,2021-02-28 01:00:00+00:00,53.11,47.03,739.95,3,0.965926,0.527078,3.263746e-07,-4.324696,-0.385859,...,2.732664e-20,-0.736348,1.302933,22.306173,34.10,-9.71,7.00,900.35,14.25,42.795219
149,2021-02-28 01:30:00+00:00,48.58,47.03,360.45,4,0.965926,0.527078,3.263746e-07,-4.324696,-0.386198,...,2.732664e-20,-0.782117,1.315346,22.304271,33.50,-9.71,7.00,900.35,14.25,42.795143
150,2021-02-28 02:00:00+00:00,47.27,42.32,422.10,5,0.866025,0.527078,3.263746e-07,-4.324696,-0.385428,...,2.732664e-20,-0.782503,1.310671,22.236828,35.05,-9.71,7.00,900.35,13.69,42.780838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,83.91,84.41,1201.85,2,0.965926,-0.591261,-1.062577e+01,33.503429,61.177328,...,1.417731e+02,173.530786,203.292972,252.058557,66.01,91.88,126.00,1271.20,50.21,91.831215
42281,2023-08-26 00:00:00+00:00,82.13,79.96,1066.95,3,1.000000,-0.577292,-3.847346e+00,7.954975,32.917332,...,9.198749e+01,109.441426,127.084166,187.448090,66.01,90.64,90.64,1643.15,37.82,87.033623
42282,2023-08-26 00:30:00+00:00,78.76,79.96,997.60,4,1.000000,-0.577292,-1.064645e+01,3.459110,29.465151,...,8.366926e+01,100.222843,120.521524,176.329598,66.01,95.15,115.99,1599.25,37.82,87.802376
42283,2023-08-26 01:00:00+00:00,77.47,73.84,913.25,5,0.965926,-0.577292,-9.140216e+00,5.808364,25.305676,...,7.956133e+01,96.108589,116.482859,160.393853,66.01,91.30,125.00,1619.70,31.66,83.545006


In [104]:
df_bbidding1.to_csv('bidding_training_predictions_day_ahead_price.csv', index=False)